In [1]:
%cd ~/gymnasium-test/
%load_ext line_profiler
%load_ext memory_profiler
%load_ext autoreload
%autoreload 2

/rdcw/fs2/home1/Active/home/n.saumik/gymnasium-test


In [1]:
from tree import Tree

importing numpy
importing tensorflow
done with imports


2024-10-24 08:08:18.881356: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [16]:
t = Tree(5, "float")

In [18]:
t.arr.round(2)

array([ 0.  ,  0.  , -1.96,  1.02,  0.  ,  0.  ,  0.61,  0.73,  0.  ,
        1.94,  0.  ,  1.  ,  0.  ,  0.  ,  0.73,  0.18,  0.86,  1.54,
       -1.7 ,  0.42,  0.03,  1.12, -1.33,  0.65,  0.62,  1.38, -1.  ,
        1.98, -1.31,  0.65,  0.69])

In [5]:
import os

print("importing tensorflow")
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["MPLCONFIGDIR"] = "/tmp/"
import tensorflow as tf

tf.config.set_visible_devices([], "GPU")
print("finished importing tensorflow")

importing tensorflow
finished importing tensorflow


In [6]:
import numpy as np
from tqdm import tqdm
import gc
import time

from tree import make_model, EarlyStopper, Forest, get_output, write_output

In [2]:
import numpy as np

In [65]:
%%time
a = np.random.default_rng(0).uniform(0, 2, size=(100000000, 6))
a[:, (1,3,5)] += 1

CPU times: user 5.29 s, sys: 1.27 s, total: 6.56 s
Wall time: 6.57 s


In [66]:
%%time
paths = np.array([a[:, p].sum(axis=1) for p in [(0,2),(0,3),(1,4),(1,5)]])

CPU times: user 4.78 s, sys: 2.22 s, total: 7 s
Wall time: 7.02 s


In [67]:
paths.max(axis=0).mean()

4.155586252617806

In [8]:
def exp(
    seed,
    decisions,
    window,
    mode,
    skill_str,
    budget,
    patience=20,
    write=True,
    opt="sgd",
    lr=1,
    num_nodes=16,
    num_layers=1,
):
    depth = decisions + 1
    print(f"\n\n\nStarting experiment: Seed-{seed} Depth-{depth} Skill-{skill_str}")

    import tensorflow as tf

    st = time.perf_counter()
    model = make_model(depth, window, opt, lr, num_nodes, num_layers)
    best = model.get_weights()

    if decisions == 4:
        if isinstance(budget, list):
            budget = np.array(budget).astype(int)
            budget_str = "_".join(str(i) for i in budget)

        elif budget.startswith("right_"):
            budget_str = budget
            budget = int(budget_str.split("_")[1]) * np.array([0.125, 0.125, 0.25, 0.5])

        elif budget.startswith("equal_"):
            budget_str = budget
            budget = int(budget_str.split("_")[1]) * np.array([0.25, 0.25, 0.25, 0.25])

        elif budget.startswith("left_"):
            budget_str = budget
            budget = int(budget_str.split("_")[1]) * np.array([0.5, 0.25, 0.125, 0.125])

        budget = budget.astype(int)
        budget = dict(zip("ABCD", budget))
        print(budget)

    print(
        f"make model: {seed} {depth} {skill_str} {budget_str}", time.perf_counter() - st
    )

    skills = [[(s, budget[s]) for s in stage] for stage in skill_str.split("_")]
    print(skills)

    stopper = EarlyStopper(patience, minimize=False)
    best_test = -np.inf

    for stagei, stage in enumerate(skills):
        print(stage)
        model = make_model(model=model)
        model.set_weights(best)

        X_trains, Y_trains, X_tests, Y_tests = [], [], [], []

        for s, budget in stage:
            st = time.perf_counter()
            train = Forest(depth, mode, budget * seed, budget * (seed + 1))
            X_train, Y_train = train.get_training_data(s, window=window)
            print(f"train data: {seed} {depth} {s} {budget}", time.perf_counter() - st)

            st = time.perf_counter()
            test_budget, test_offset = 2048, int(1e9)
            test = Forest(
                depth,
                mode,
                test_budget * seed + test_offset,
                test_budget * (seed + 1) + test_offset,
            )
            X_test, Y_test = test.get_training_data(s, window=window)
            print(f"test data: {seed} {depth} {s} {budget}", time.perf_counter() - st)
            X_trains.append(X_train)
            Y_trains.append(Y_train)
            X_tests.append(X_test)
            Y_tests.append(Y_test)

        X_train = tf.convert_to_tensor(np.concatenate(X_trains))
        Y_train = tf.convert_to_tensor(np.concatenate(Y_trains))
        X_test = tf.convert_to_tensor(np.concatenate(X_tests))
        Y_test = tf.convert_to_tensor(np.concatenate(Y_tests))

        tqdm_kwargs = {
            "ncols": 80,
            "leave": True,
            "desc": f"Seed {seed} Depth {depth} Stage {'+'.join(s for s, _ in stage)}",
            # "position": 1,
            # "disable": True,
        }

        stopper.wait = 0
        for epoch in tqdm(range(2000), **tqdm_kwargs):
            # print()
            # st = time.perf_counter()
            if epoch != 0:
                model.fit(X_train, Y_train, batch_size=256, verbose=0)
            # print(f"model fit: {seed} {epoch}", time.perf_counter() - st)

            # st = time.perf_counter()
            loss, acc = model.evaluate(X_test, Y_test, batch_size=4096, verbose=0)
            # print(f"model evaluate: {seed} {epoch}", time.perf_counter() - st)

            # st = time.perf_counter()
            _, scores, _ = test.eval_model(model, window=window)
            valid_score = np.mean(scores[: test_budget // 2])
            test_score = np.mean(scores[test_budget // 2 :])
            # print(f"model eval_model: {seed} {epoch}", time.perf_counter() - st)
            at_best, at_patience = stopper.should_stop(valid_score)
            if at_best:
                best = model.get_weights()
                best_output = get_output(
                    seed,
                    depth - 1,
                    window,
                    mode,
                    stagei,
                    skill_str,
                    budget_str,
                    opt,
                    lr,
                    num_nodes,
                    num_layers,
                    epoch,
                    loss,
                    acc,
                    valid_score,
                    stopper.wait,
                    stopper.patience,
                    test_score,
                )
                best_test = test_score

            if at_patience:
                write_output(best_output)
                break

        st = time.perf_counter()
    del X_train, Y_train, X_test, Y_test
    del train, test
    del model
    tf.keras.backend.clear_session()
    gc.collect()
    print(
        f"deleting: {seed} {depth} {skill_str} {budget_str}", time.perf_counter() - st
    )
    return stopper.best_value, best_test

: 

In [9]:
# decisions = int(sys.argv[2])
# assert decisions == 4
# mode = sys.argv[3]
# budget_str = int(sys.argv[4])
# window = 0
# start, end = [int(i) for i in sys.argv[5].split("-")]

decisions = 4
assert decisions == 4
mode = "normal-2-0"
budget_str = int(2)
window = 0
start, end = 0, 1
skills = "ABCD"[:decisions]
onepass = "_".join(skills)


for seed in range(start, end):

    budget = [2, 2, 2, 2]
    while sum(budget) < 2**18:
        cur = sum(budget)
        branch_add = cur // 8
        branch_vals = {}
        for i in range(4):
            branch_budget = budget.copy()
            branch_budget[i] += branch_add
            valid_score, test_score = exp(
                seed,
                decisions,
                window,
                mode,
                onepass,
                branch_budget,
                write=False,
            )
            branch_vals[tuple(branch_budget)] = valid_score, test_score
        print(branch_vals)
        break

: 